In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv("../data/data.csv")
df = df.drop("Unnamed: 0", axis = 1)

In [11]:
df.head(2)

,address,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,...,RestaurantsCounterService,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTableService,RestaurantsTakeOut,Smoking,WheelchairAccessible,WiFi
0,30 Eglinton Avenue W,QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,...,NaN,False,True,2,True,True,True,NaN,NaN,u'no'
1,"10110 Johnston Rd, Ste 15",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,...,NaN,False,True,2,True,True,True,NaN,NaN,u'no'


In [16]:
df["stars"].unique()

array([2.5, 4. , 3.5, 4.5, 2. , 3. , 5. , 1.5, 1. ])

In [13]:
df.groupby("city")["business_id"].count().sum()

108179

In [17]:
df.name.value_counts()

Starbucks                          955
McDonald's                         641
Subway                             518
Tim Hortons                        300
Walgreens                          285
                                  ... 
Lyna Nails                           1
Tango Vietnamese Tropical Grill      1
Teashop 168                          1
Toasties                             1
Eve's Tease                          1
Name: name, Length: 56462, dtype: int64

In [15]:
df = df[df["is_open"] == 1]